In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/d/gauravduttakiit/resume-dataset/UpdatedResumeDataSet.csv


In [9]:
!pip install -q scikit-learn pandas spacy

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
import re
import spacy

# Load spaCy model
import en_core_web_sm
nlp = en_core_web_sm.load()

In [10]:
df = pd.read_csv('/kaggle/input/d/gauravduttakiit/resume-dataset/UpdatedResumeDataSet.csv') 
print(df['Category'].value_counts())

Category
Java Developer               84
Testing                      70
DevOps Engineer              55
Python Developer             48
Web Designing                45
HR                           44
Hadoop                       42
Sales                        40
Data Science                 40
Mechanical Engineer          40
ETL Developer                40
Blockchain                   40
Operations Manager           40
Arts                         36
Database                     33
Health and fitness           30
PMO                          30
Electrical Engineering       30
Business Analyst             28
DotNet Developer             28
Automation Testing           26
Network Security Engineer    25
Civil Engineer               24
SAP Developer                24
Advocate                     20
Name: count, dtype: int64


In [11]:
def clean_text(text):
    # Remove unwanted characters
    text = re.sub(r'\s+', ' ', text)             # Remove extra spaces
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)    # Keep only alphanumerics
    text = text.lower()

    # Lemmatization with spaCy
    doc = nlp(text)
    text = ' '.join([token.lemma_ for token in doc if not token.is_stop])
    return text

In [12]:
# Clean the resume texts
df['Cleaned_Resume'] = df['Resume'].apply(clean_text)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df['Cleaned_Resume'], df['Category'], test_size=0.2, random_state=42
)

In [13]:
# TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=3000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [14]:
# Train Logistic Regression Model
model = LogisticRegression(max_iter=200)
model.fit(X_train_tfidf, y_train)

# Predictions and Evaluation
y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9948186528497409

Classification Report:
                            precision    recall  f1-score   support

                 Advocate       1.00      1.00      1.00         3
                     Arts       1.00      1.00      1.00         6
       Automation Testing       1.00      1.00      1.00         5
               Blockchain       1.00      1.00      1.00         7
         Business Analyst       1.00      1.00      1.00         4
           Civil Engineer       1.00      1.00      1.00         9
             Data Science       1.00      1.00      1.00         5
                 Database       1.00      1.00      1.00         8
          DevOps Engineer       1.00      0.93      0.96        14
         DotNet Developer       1.00      1.00      1.00         5
            ETL Developer       1.00      1.00      1.00         7
   Electrical Engineering       1.00      1.00      1.00         6
                       HR       1.00      1.00      1.00        12
       

In [15]:
job_description = """
Looking for a Data Scientist with experience in Python, machine learning, deep learning, and natural language processing.
"""

In [16]:

# Clean the job description
cleaned_job_desc = clean_text(job_description)

# Vectorize the job description using the same TF-IDF vectorizer
job_desc_vector = vectorizer.transform([cleaned_job_desc])

# Vectorize all resumes
resume_vectors = vectorizer.transform(df['Cleaned_Resume'])

# Calculate cosine similarity between job description and each resume
similarities = cosine_similarity(job_desc_vector, resume_vectors).flatten()

# Get top 10 matching resumes
top_10_indices = similarities.argsort()[-10:][::-1]
top_10_resumes = df.iloc[top_10_indices]

# Output
print("Top 10 Matching Resumes for the Job Description:\n")
for idx, row in top_10_resumes.iterrows():
    print(f"\n📌 Candidate {idx+1}")
    print(f"Category: {row['Category']}")
    print(f"Similarity Score: {similarities[idx]:.4f}")
    print(f"Resume Snippet: {row['Resume'][:300]}...\n")


Top 10 Matching Resumes for the Job Description:


📌 Candidate 28
Category: Data Science
Similarity Score: 0.3889
Resume Snippet: Education Details 
 B.Tech   Rayat and Bahra Institute of Engineering and Biotechnology
Data Science 

Data Science
Skill Details 
Numpy- Exprience - Less than 1 year months
Machine Learning- Exprience - Less than 1 year months
Tensorflow- Exprience - Less than 1 year months
Scikit- Exprien...


📌 Candidate 38
Category: Data Science
Similarity Score: 0.3889
Resume Snippet: Education Details 
 B.Tech   Rayat and Bahra Institute of Engineering and Biotechnology
Data Science 

Data Science
Skill Details 
Numpy- Exprience - Less than 1 year months
Machine Learning- Exprience - Less than 1 year months
Tensorflow- Exprience - Less than 1 year months
Scikit- Exprien...


📌 Candidate 18
Category: Data Science
Similarity Score: 0.3889
Resume Snippet: Education Details 
 B.Tech   Rayat and Bahra Institute of Engineering and Biotechnology
Data Science 

Data Science
Sk